In [1]:
import tensorflow as tf
import cv2
import os
import pathlib
import time
import datetime
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
from PIL import Image

In [2]:
os.chdir(r'C:\Users\jared\Deblur Project\input')
os.curdir
init_size = 100              #process 100 files at a time
folders = os.listdir()
filelist=[]                  #Keeps track of files.
blurry_data=[]               #stores the blurred images.
sharp_data=[]                #stores the sharp image.


#filling the blurry data set
blurry_files = os.listdir(folders[0])
os.chdir(r'C:\Users\jared\Deblur Project\input\Gaussian_blurred')
for i in blurry_files:
    im = Image.open(i)             #opening each image
    width = 256   #setting width of image
    height = 256 #setting height of image
    im5 = im.resize((width, height), Image.ANTIALIAS)   #resizing image
    x = np.asarray(im5)             #convert image to array
   # x =(x-x.mean())/255.0# best down-sizing filter
    blurry_data.append(x)


os.chdir(r'C:\Users\jared\Deblur Project\input')

#filling the sharp data set
sharp_files = os.listdir(folders[2])
os.chdir(r'C:\Users\jared\Deblur Project\input\sharp')
for i in sharp_files:
    im = Image.open(i)             #opening each image
    width = 256   #setting width of image
    height = 256 #setting height of image
    im5 = im.resize((width, height), Image.ANTIALIAS)   #resizing image
    x = np.asarray(im5)             #convert image to array
    #x =(x-x.mean())/255.0# best down-sizing filter
    sharp_data.append(x)
    
blurry_data = np.asarray(blurry_data)
sharp_data = np.asarray(sharp_data)

In [3]:
def concat_lib(image_set, how, with_plot=False):
    # dimension of each matrix in image_set
    shape_vals = [imat.shape for imat in image_set]
    # length of dimension of each matrix in image_set
    shape_lens = [len(ishp) for ishp in shape_vals]
    # if all the images in image_set are read in same mode
    channel_flag = True if len(set(shape_lens)) == 1 else False

    if channel_flag:
        ideal_shape = max(shape_vals)
        images_resized = [
            # function call to resize the image
            resize_image(image_matrix=imat, nh=ideal_shape[0], nw=ideal_shape[1]) 
            if imat.shape != ideal_shape else imat for imat in image_set
        ]
    else:
        return False

    # cv2 library code to concatenate the image matrices
    # we use methods like
    #     - vconcat() → vertical concat
    #     - hconcat() → horizontal concat
    if (how == 0) or (how == 'vertical'):
        concats = cv2.vconcat(images_resized)
    elif (how == 1) or (how == 'horizontal'):
        concats = cv2.hconcat(images_resized)
    else:
        concats = cv2.hconcat(images_resized)
    if with_plot:
        cmap_val = None if len(concats.shape) == 3 else 'gray'
        plt.figure(figsize=(10, 6))
        plt.axis("off")
        plt.imshow(concats, cmap=cmap_val)
        return True
    return concats

In [4]:
destination = r'C:\Users\jared\Deblur Project\concatenated images/filename'
for i in range(0, len(blurry_data)):
    c_image = concat_lib(image_set = [sharp_data[i], blurry_data[i]], how = 'horizontal', with_plot = False)
    plt.imsave(destination + str(i) +'.jpg', c_image)